In [2]:
#load up the basics
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd

#Set GPU to use
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


#image file issue
#https://stackoverflow.com/questions/48395076/keras-giving-oserror-image-file-is-truncated-26-bytes-not-processed
from PIL import ImageFile, Image
ImageFile.LOAD_TRUNCATED_IMAGES = True
Image.MAX_IMAGE_PIXELS = None

#import TF stuff
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#Check TensorFlow Version
print('TF version: {}' .format(tf.__version__))

#Check for GPU utilization
if tf.test.gpu_device_name():
    print(' GPU: {}'.format(tf.test.gpu_device_name()))
else:
    print("No GPU")

TF version: 2.9.1
 GPU: /device:GPU:0


In [3]:
#Load in DF
dfpath = '../data/TestingPreds.csv'
df = pd.read_csv(dfpath)

df.head()

,Unnamed: 0,doodled_image,imageP,image,Filename,storm_id,archive,Latitude,Longitude,is_unique,doodled_only_once,doodled_image_path,doodled_label_path,DiceLoss,mDice,mIoU,IoU_c1,IoU_c2,IoU_c3,IoU_c4
0,0,S21775624-0_L1.jpg,S21775624-0.jpg,S21775624.jpg,Isaias/20200804a_jpgs/jpgs/S21775624.jpg,isaias,20200804a_jpgs,32.914668,-79.574800,True,True,../data/testing/images/S21775624-0_L1.jpg,../data/testing/labels/S21775624-0_L1_label.jpg,0.658302,0.341698,0.253098,0.339408,0.620179,0.048835,0.003968
1,1,S21767442-2_L1.jpg,S21767442-2.jpg,S21767442.jpg,Isaias/20200804a_jpgs/jpgs/S21767442.jpg,isaias,20200804a_jpgs,34.661264,-76.496155,True,True,../data/testing/images/S21767442-2_L1.jpg,../data/testing/labels/S21767442-2_L1_label.jpg,0.750014,0.249986,0.499957,0.999915,0.000000,0.000000,0.000000
2,2,P19747944-0_L1.jpg,P19747944-0.jpg,P19747944.jpg,Barry/20190716a_jpgs/jpgs/P19747944.jpg,barry,20190716a_jpgs,29.729276,-93.013003,True,True,../data/testing/images/P19747944-0_L1.jpg,../data/testing/labels/P19747944-0_L1_label.jpg,0.354577,0.645423,0.572283,0.886405,0.705786,0.696941,0.000000
3,3,S28474304-0_L1.jpg,S28474304-0.jpg,S28474304.jpg,Michael/20181011a_jpgs/jpgs/S28474304.jpg,michael,20181011a_jpgs,30.240322,-87.720852,True,True,../data/testing/images/S28474304-0_L1.jpg,../data/testing/labels/S28474304-0_L1_label.jpg,0.307423,0.692577,0.578349,0.908507,0.545584,0.174192,0.685114
4,4,P26056634-0_L1.jpg,P26056634-0.jpg,P26056634.jpg,Florence/20180917a_jpgs/jpgs/P26056634.jpg,florence,20180917a_jpgs,34.553545,-77.294043,True,True,../data/testing/images/P26056634-0_L1.jpg,../data/testing/labels/P26056634-0_L1_label.jpg,0.371731,0.628269,0.554010,0.897764,0.509506,0.808769,0.000000


In [4]:
#Load up the TF model

#model path
filepath = './model/PredictIoU'

#load model
model = tf.keras.models.load_model(filepath, compile = True)

In [5]:
# standardization using adjusted standard deviation, as required by Gym model

def standardize(img):

    N = np.shape(img)[0] * np.shape(img)[1]
    s = np.maximum(np.std(img), 1.0 / np.sqrt(N))
    m = np.mean(img)
    img = (img - m) / s
    del m, s, N
    #
    if np.ndim(img) == 2:
        img = np.dstack((img, img, img))

    return img


In [6]:
#run model for each row of dataframe, and put results in correct column

#Evaluate on all images:
# 1. loop over all rows in data frame
# 2. grab image path
# 3. evaluate image w/ TFLite
# 4. write output to df
# 5. make plot

# fn to do tflite prediction
def TFPred (ImPath):
    imsize = (768,1024)
    #SET THE IMAGE
    TestIm = tf.keras.preprocessing.image.load_img(ImPath,target_size = imsize)
    TestIm = tf.keras.preprocessing.image.img_to_array(TestIm)
    TestIm = standardize(TestIm)
    TestIm = np.expand_dims(TestIm,axis=0)
    
    #run model
    results = model.predict(TestIm)
    
    return results

#mmake an empty dictionary to hold predictions
pred_dict = []

## loop over all rows in data frame, grab image path, evaluate image w/ TFLite helper fn, 
# then flatten results adn append ot the dictionary
for index, row in df.iterrows():
    imgpath = row['doodled_image_path'] 
    pred = TFPred(imgpath)
    pred_dict.append(pred)
    #print(pred)
    print(index)



1/1 [==============================] - 2s 2s/step
0
1/1 [==============================] - 0s 18ms/step
1
1/1 [==============================] - 0s 17ms/step
2
1/1 [==============================] - 0s 18ms/step
3
1/1 [==============================] - 0s 18ms/step
4
1/1 [==============================] - 0s 19ms/step
5
1/1 [==============================] - 0s 17ms/step
6
1/1 [==============================] - 0s 17ms/step
7
1/1 [==============================] - 0s 20ms/step
8
1/1 [==============================] - 0s 17ms/step
9
1/1 [==============================] - 0s 17ms/step
10
1/1 [==============================] - 0s 17ms/step
11
1/1 [==============================] - 0s 17ms/step
12
1/1 [==============================] - 0s 17ms/step
13
1/1 [==============================] - 0s 17ms/step
14
1/1 [==============================] - 0s 17ms/step
15
1/1 [==============================] - 0s 14ms/step
16
1/1 [==============================] - 0s 17ms/step
17
1/1 [=================

In [7]:
#turn list of arrays into dataframe, add columns, and add to the test df
predictions = pd.DataFrame(np.concatenate(pred_dict))
predictions.columns=[ "PmDice", "PmIoU", "PIoU_c1","PIoU_c2","PIoU_c3", "PIoU_c4"]
predictions.head()

df = df.join(predictions)

df.head()

,Unnamed: 0,doodled_image,imageP,image,Filename,storm_id,archive,Latitude,Longitude,is_unique,...,IoU_c1,IoU_c2,IoU_c3,IoU_c4,PmDice,PmIoU,PIoU_c1,PIoU_c2,PIoU_c3,PIoU_c4
0,0,S21775624-0_L1.jpg,S21775624-0.jpg,S21775624.jpg,Isaias/20200804a_jpgs/jpgs/S21775624.jpg,isaias,20200804a_jpgs,32.914668,-79.574800,True,...,0.339408,0.620179,0.048835,0.003968,0.393231,0.300356,0.508620,0.490042,0.098742,0.046255
1,1,S21767442-2_L1.jpg,S21767442-2.jpg,S21767442.jpg,Isaias/20200804a_jpgs/jpgs/S21767442.jpg,isaias,20200804a_jpgs,34.661264,-76.496155,True,...,0.999915,0.000000,0.000000,0.000000,0.454404,0.571095,0.990940,0.663109,0.000000,0.000000
2,2,P19747944-0_L1.jpg,P19747944-0.jpg,P19747944.jpg,Barry/20190716a_jpgs/jpgs/P19747944.jpg,barry,20190716a_jpgs,29.729276,-93.013003,True,...,0.886405,0.705786,0.696941,0.000000,0.479027,0.420606,0.640798,0.628373,0.395551,0.000000
3,3,S28474304-0_L1.jpg,S28474304-0.jpg,S28474304.jpg,Michael/20181011a_jpgs/jpgs/S28474304.jpg,michael,20181011a_jpgs,30.240322,-87.720852,True,...,0.908507,0.545584,0.174192,0.685114,0.888756,0.755034,1.032883,0.661543,0.605497,0.813640
4,4,P26056634-0_L1.jpg,P26056634-0.jpg,P26056634.jpg,Florence/20180917a_jpgs/jpgs/P26056634.jpg,florence,20180917a_jpgs,34.553545,-77.294043,True,...,0.897764,0.509506,0.808769,0.000000,0.648091,0.551261,0.723717,0.611928,0.644833,0.218140


In [9]:
df.to_csv('../data/Test_Metrics_Preds.csv')

In [11]:
df["se_mDice"] = (df["PmDice"] - df["mDice"]) ** 2
df["se_mIoU"] = (df["PmIoU"] - df["mIoU"]) ** 2
df["se_IoU_c1"]= (df["PIoU_c1"] - df["IoU_c1"]) ** 2
df["se_IoU_c2"]= (df["PIoU_c1"] - df["IoU_c1"]) ** 2
df["se_IoU_c3"]= (df["PIoU_c1"] - df["IoU_c1"]) ** 2
df["se_IoU_c4"]= (df["PIoU_c1"] - df["IoU_c1"]) ** 2


print(df["se_mDice"].mean())
print(df["se_mIoU"].mean())
print(df["se_IoU_c1"].mean())
print(df["se_IoU_c2"].mean())
print(df["se_IoU_c3"].mean())
print(df["se_IoU_c4"].mean())


0.020179301955948786
0.020291981054224507
0.03464828425862954
0.03464828425862954
0.03464828425862954
0.03464828425862954
